<a href="https://colab.research.google.com/github/sean-otoole/2p_neural_activity_otoole_2023/blob/main/initial_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/iob_interview_prep")

!pwd

Mounted at /content/drive


In [9]:
!curl -L https://datasets.cellxgene.cziscience.com/45015077-9d98-467d-a0ff-55d04e7e47e8.h5ad -o /content/drive/MyDrive/iob_interview_prep/human_retina.h5ad

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.6G  100 17.6G    0     0  40.8M      0  0:07:23  0:07:23 --:--:-- 44.9M


In [10]:
# multi omics data from "Single Cell Multiome Atlas of the Human Fetal Retina"
# from here: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE268630

# single cell adult atlas from: "A multi-omics atlas of the human retina at single-cell resolution"
# data from here: "https://explore.data.humancellatlas.org/projects/9c20a245-f2c0-43ae-82c9-2232ec6b594f"
# downloaded here: "https://cellxgene.cziscience.com/collections/af893e86-8e9f-41f1-a474-ef05359b1fb7"